Bu ödevinizde verilen ratings.csv dosyasını kullanacaksınız. Bu dosyada kullanıcıların farklı ürünler için verdikleri puanlar vardır. En yüksek puan 10'dur.

Ödevinizde pandas, numpy kütüphanelerini kullanabilirsiniz. Ayrıca grafik oluşturmak için herhangi bir görselleştirme kütüphanesi kullanabilirsiniz. Bunlar dışında herhangi bir kütüphane kullanamazsınız.

Tüm yakınlık hesaplamaları için kosinus benzerliğini kullanınız.

**Sorularda belirtilmeyen bir parametre vs. var ise, o kısım sizin dizayn kararınıza bırakılmıştır demektir. Bu durumlarda aldığınız kararı belirtiniz.**

**Soru 1 [5 puan]** İlgili veri kümesinin “ratings.csv” verisinde satır sayısının mod 10’u sıfır olan satırlar test için gerisi eğitim için kullanılacaktır. Yani, 10. 20., 30., … satırlar test için kullanılacaktır. Eğitim ve test kümelerini belirleyiniz. Satırları saymaya 0'dan başlayınız.

In [1]:
import pandas as pd
import numpy as np

ratings = pd.read_csv('Ratings.csv')

In [2]:
ratings.head(5)

,User-ID,ISBN,Book-Rating,Price
0,276725,034545104X,0,74
1,276726,155061224,5,37
2,276727,446520802,0,95
3,276729,052165615X,3,37
4,276729,521795028,6,59


In [3]:
test = ratings.iloc[9::10, :]
train = ratings.drop(ratings.index[9::10])

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

**Soru 2 [20 puan]** Item bazlı “Collaborative filtering” yöntemini kullanarak en yakın 5 komşuyu göz önünde alarak test kümesinin ilk beş değeri için yapılan tahmin değerlerini ve gerçek değeri gösteriniz. 

In [4]:
import numpy as np

def calculate_similarity(main_vec, other_vec):
    fill_value = np.nan
    filled_main = [fill_value if np.isnan(x) else x for x in main_vec]
    main_vec = np.array(filled_main)

    filled_other = [fill_value if np.isnan(x) else x for x in other_vec]
    other_vec = np.array(filled_other)
    
    main_vec_indices = np.where(~np.isnan(main_vec))[0]
    other_vec_indices = np.where(~np.isnan(other_vec))[0]

    common_indices = np.intersect1d(main_vec_indices, other_vec_indices)

    if common_indices.size == 0:
        return 0

    main_mean = np.nanmean(main_vec)
    other_mean = np.nanmean(other_vec)

    main_vec_common = main_vec[common_indices] - main_mean
    other_vec_common = other_vec[common_indices] - other_mean

    dot_product = np.dot(main_vec_common, other_vec_common)
    norm_main = np.linalg.norm(main_vec_common)
    norm_other = np.linalg.norm(other_vec_common)

    if norm_main == 0:
        norm_main = 0.001
    if norm_other == 0:
        norm_other = 0.001

    similarity = dot_product / (norm_main * norm_other)
    
    return similarity

In [5]:
print("Train shape :", train.shape)
print("User-ID unique :", train['User-ID'].nunique())
print("ISBN unique :", train['ISBN'].nunique())

Train shape : (943718, 4)
User-ID unique : 90101
ISBN unique : 303012


In [6]:
test.head(5)

,User-ID,ISBN,Book-Rating,Price
0,276745,342310538,10,69
1,276747,671537458,9,31
2,276755,451166892,5,13
3,276762,3404611306,0,72
4,276762,3453213025,3,10


**Soru 3 [20 puan]** Kullanıcı bazlı “Collaborative filtering” yöntemini kullanarak en yakın 5 komşuyu göz önünde alarak test kümesinin ilk beş değeri için yapılan tahmin değerlerini ve gerçek değeri gösteriniz.

In [16]:
def User_user_filter(data, user_id, item_id, k=10, shift = 1000):

    utility_matrix = None
    if item_id not in data['ISBN'].values:
        print("Item not found!")
        return

    filtered_data = data[data['ISBN'] == item_id]
    user_ids_with_isbn = filtered_data['User-ID'].tolist()
    user_ids_with_isbn.append(user_id)

    temp_data = data[data['User-ID'].isin(user_ids_with_isbn)]
    del filtered_data

    temp_data.drop_duplicates(subset=['User-ID', 'ISBN'], inplace=True)

    if user_id not in temp_data['User-ID'].values:
        print("User not found!")
        return 0

    import pandas as pd

    utility_matrix = pd.DataFrame()

    for i in range(0, temp_data.shape[0], shift):
        chunk = temp_data.iloc[i:i + shift, :]
        chunk_table = chunk.groupby(['User-ID', 'ISBN'])['Book-Rating'].max().unstack(fill_value=np.nan)
        chunk_table.index.name = None
        chunk_table.columns.name = None
        sparse_dtype = pd.SparseDtype(float, fill_value=np.nan)
        chunk_table = chunk_table.astype(sparse_dtype)

        # Combine chunk_table with utility_matrix using vectorized operation
        utility_matrix = utility_matrix.combine_first(chunk_table)

        utility_matrix = utility_matrix.loc[:, ~utility_matrix.columns.duplicated()]
        utility_matrix = utility_matrix.astype(sparse_dtype)


    del temp_data
    main_vec = utility_matrix.loc[user_id].values

    utility_matrix['similarity'] = utility_matrix.apply(lambda x: calculate_similarity(main_vec, x.values), axis=1)

    utility_matrix.drop(user_id, axis=0, inplace=True)
    most_similar_users = utility_matrix.sort_values(by='similarity', ascending=False).index[:k+1]

    similarities = utility_matrix.loc[most_similar_users]['similarity']
    ratings = utility_matrix.loc[most_similar_users][item_id]

    if np.dot(ratings, similarities) == 0:
        return 0
    
    rec_score = round(np.dot(ratings, similarities) / similarities.sum(),2)

    return rec_score

In [8]:
test.head(5)

,User-ID,ISBN,Book-Rating,Price
0,276745,342310538,10,69
1,276747,671537458,9,31
2,276755,451166892,5,13
3,276762,3404611306,0,72
4,276762,3453213025,3,10


In [10]:
a = User_user_filter(train, 276745 , '342310538', k= 5)
print('Gerçek Değer :' ,test.iloc[0]['Book-Rating'])
print('Tahmin Değeri :' ,a)

User not found!
Gerçek Değer : 10
Tahmin Değeri : 0


In [9]:
a = User_user_filter(train, 276747 , '671537458', k= 5)
print('Gerçek Değer :' ,test.iloc[0]['Book-Rating'])
print('Tahmin Değeri :' ,a)

C:\Users\Emre\AppData\Local\Temp\ipykernel_4120\250329578.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data.drop_duplicates(subset=['User-ID', 'ISBN'], inplace=True)


Gerçek Değer : 10
Tahmin Değeri : 1.3333333333333333


In [14]:
a = User_user_filter(train, 276755 , '451166892', k= 5)
print('Gerçek Değer :' ,test.iloc[0]['Book-Rating'])
print('Tahmin Değeri :' ,a)

User not found!
Gerçek Değer : 10
Tahmin Değeri : 0


C:\Users\Emre\AppData\Local\Temp\ipykernel_24960\730667034.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data.drop_duplicates(subset=['User-ID', 'ISBN'], inplace=True)


In [13]:
a = User_user_filter(train, 276762 , '3404611306', k= 5)
print('Gerçek Değer :' ,test.iloc[0]['Book-Rating'])
print('Tahmin Değeri :' ,a)

C:\Users\Emre\AppData\Local\Temp\ipykernel_24960\730667034.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data.drop_duplicates(subset=['User-ID', 'ISBN'], inplace=True)


Gerçek Değer : 10
Tahmin Değeri : 0


In [17]:
a = User_user_filter(train, 276762 , '3453213025', k= 5)
print('Gerçek Değer :' ,test.iloc[0]['Book-Rating'])
print('Tahmin Değeri :' ,a)

Item not found!
Gerçek Değer : 10
Tahmin Değeri : None


**Soru 4 [15 puan]**  Item bazlı “Collaborative filtering” yöntemini kullanarak en yakın komşu sayısı 1, 3, 5, ve 7 olduğunda test kümesi üzerinde RMSE değerlerini hesaplayınız ve sonuçlarınızı bir figür ya da tablo gibi açıkca anlaşılacak bir şekilde gösteriniz.

In [ ]:
# cevabınızı buraya yazacaksınız

**Soru 5 [15 puan]**  Kullanıcı bazlı “Collaborative filtering” yöntemini kullanarak en yakın komşu sayısı 1, 3, 5, ve 7 olduğunda test kümesi üzerinde RMSE değerlerini hesaplayınız ve sonuçlarınızı bir figür ya da tablo gibi açıkca anlaşılacak bir şekilde gösteriniz.

In [ ]:
# cevabınızı buraya yazacaksınız

**Soru 6 [25 puan]**  Kullanıcıya döndüğünüz ürünlerden ilk 5'inde 9 veya 10 puan olan ürünleri satın alacak. 

a) Bir kullanıcı maksimum 5 ürün alacaksa, test kümesinde maksimum toplam satış kaç olabilir?


b) Önceki sorularda geliştirdiğiniz kullanıcı bazlı “Collaborative filtering” ve Item bazlı “Collaborative filtering” yöntemlerine göre firmanın kazanacağı parayı hesaplayınız. “Collaborative filtering” yöntemlerinde bakılacak en yakın komşu sayısı 5'tir.

c) Firmanın kazandığı parayı da arttıracak şekilde Item bazlı “Collaborative filtering” yöntemini geliştiriniz. Önerdiğiniz yöntem ile ne kadar kazanç olacağını hesaplayınız.

In [ ]:
# cevabınızı buraya yazacaksınız